# 08: 正則化と実践的テクニック (Regularization & Advanced Techniques)

このノートブックでは、ニューラルネットワークの汎化性能を高め、学習を安定させるための2つの重要なテクニック、「Dropout」と「Gradient Clipping」について学びます。
これまでのノートブックで学んだ技術（活性化関数、初期化、オプティマイザ、正規化層）は、主にモデルが訓練データにうまく適合し、学習を高速化・安定化させることを目的としていました。

今回は、モデルが訓練データに過度に適合してしまう「過学習（Overfitting）」を防ぎ、未知のデータに対しても高い性能を発揮する（汎化する）ための代表的な手法である**Dropout**と、RNNの学習などで発生しやすい「勾配爆発」を防ぐための**Gradient Clipping**を扱います。

**参考論文:**
*   (Dropout) Srivastava, N., Hinton, G., et al. (2014). Dropout: A Simple Way to Prevent Neural Networks from Overfitting.
*   (Gradient Clipping) Pascanu, R., Mikolov, T., & Bengio, Y. (2013). On the difficulty of training recurrent neural networks.

**このノートブックで学ぶこと:**
1.  過学習（Overfitting）の概念とその問題点。
2.  Dropoutのアルゴリズム：学習時にランダムにニューロンを非活性化する。
3.  学習時と推論時でDropoutの振る舞いが異なる理由と、その調整（Inverted Dropout）。
4.  Dropoutがなぜ正則化として機能するのか（アンサンブル学習の観点）。
5.  勾配爆発（Gradient Exploding）の問題と、それを防ぐGradient Clippingの仕組み。

**前提知識:**
*   ニューラルネットワークの基本的な学習プロセス。
*   過学習と汎化の概念。
*   勾配降下法と勾配の役割。
*   NumPyとMatplotlibの基本的な使い方。

## 1. 必要なライブラリのインポート

In [4]:
import numpy as np
import matplotlib.pyplot as plt

## 2. 過学習 (Overfitting) とは？

過学習とは、ニューラルネットワークが**訓練データに過剰に適合してしまい、そのデータの特定のパターンやノイズまで学習してしまった結果、未知の新しいデータ（テストデータ）に対してうまく性能を発揮できなくなる現象**を指します。

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/Overfitting.svg/600px-Overfitting.svg.png" width="400"></center>
<center><small>出典: Wikimedia Commons</small></center>

上の図のように、緑の線（過学習モデル）は訓練データ（青い点）を完璧に通っていますが、真の関数（黒い線）からは大きく外れてしまっています。これでは、新しいデータポイントが来たときに大きな誤差を生んでしまいます。

過学習は、モデルの表現力（パラメータ数）がデータの複雑さに対して高すぎる場合に特に起こりやすくなります。モデルの複雑さを抑え、汎化性能を高めるためのテクニックを総称して**正則化**と呼びます。Dropoutはその代表的な手法の一つです。

## 3. Dropout

Dropoutは、2014年に提案された非常にシンプルかつ強力な正則化手法です。そのアイデアは、「**学習時に、各ニューロンを一定の確率 $p$ でランダムに非活性化（出力を0に）する**」というものです。

### 3.1 Dropoutのアルゴリズム

**学習時:**
1.  順伝播の際、ある層の各ニューロンに対して、確率 $p$ で「ドロップ（非活性化）」するかどうかを決定します。
2.  ドロップすると選ばれたニューロンの出力は0になります。
3.  ドロップされなかったニューロンの出力は、そのまま次の層に伝播します。

**推論時:**
1.  推論時には、ニューロンをドロップアウトしません。**全てのニューロンを使用します。**
2.  しかし、学習時にはニューロンの一部しか使っていなかったため、そのままだと出力のスケールが学習時と異なってしまいます。
3.  このスケールを合わせるために、各ニューロンの出力を**学習時にドロップアウトされなかった確率 $(1-p)$ でスケールダウン**(**乗算**)します。


### 3.2 Inverted Dropout（近年の標準的な実装）

推論時のスケール調整は、毎回計算が必要で少し面倒です。そこで、**Inverted Dropout**という実装が現在では標準となっています。
この手法では、スケール調整を**学習時**に行います。

**Inverted Dropoutのアルゴリズム:**
*   **学習時**:
    1.  ニューロンを確率 $p$ でドロップアウトします。
    2.  ドロップされなかったニューロンの出力を、確率 $(1-p)$ で**割る**ことでスケールアップします。
*   **推論時**:
    1.  何もしません。全てのニューロンをそのまま使用します。

学習時にあらかじめスケールを調整しておくことで、推論時の処理が不要になり、実装がシンプルになります。

**数式 (Inverted Dropout):**
層の出力を $a$、ドロップアウト率を $p$ とすると、
$$
\text{Dropout}(a) =
\begin{cases}
\frac{a}{1-p} & \text{with probability } 1-p \\
0 & \text{with probability } p
\end{cases}
$$

In [5]:
def dropout(x, dropout_ratio=0.5, training=True):
    """
    Inverted Dropoutの実装
    """
    if not training:
        return x

    # ドロップアウトするニューロンを決めるマスクを生成
    # dropout_ratioより大きい値はTrue(生存)、小さい値はFalse(ドロップ)
    mask = np.random.rand(*x.shape) > dropout_ratio
    
    # マスクを適用し、スケールを調整
    return x * mask / (1.0 - dropout_ratio)

# 簡単な実験
x = np.ones(10)
dropout_ratio = 0.5

print("Original data:", x)

# 学習時
y_train = dropout(x, dropout_ratio, training=True)
print(f"\nAfter Dropout (training=True, p={dropout_ratio}):\n", y_train)
print("-> ニューロンがランダムに0になり、生存したニューロンの出力が 1/(1-p)=2.0 にスケールされている")

# 推論時
y_test = dropout(x, dropout_ratio, training=False)
print("\nAfter Dropout (training=False):\n", y_test)
print("-> 何も変化しない")

Original data: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

After Dropout (training=True, p=0.5):
 [2. 0. 2. 2. 0. 2. 0. 0. 2. 0.]
-> ニューロンがランダムに0になり、生存したニューロンの出力が 1/(1-p)=2.0 にスケールされている

After Dropout (training=False):
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
-> 何も変化しない


### 3.3 Dropoutはなぜ機能するのか？

Dropoutが強力な正則化手法として機能する理由は、主に2つの観点から説明されます。

1.  **アンサンブル学習の効果**:
    Dropoutは、学習の各イテレーションで異なるニューロンの組み合わせを持つ「痩せた（thinned）」ネットワークを学習していると見なせます。これは、膨大な数の異なるネットワークアーキテクチャを訓練し、推論時にはそれらの予測を平均化（アンサンブル）するのと似た効果をもたらします。アンサンブル学習は、単一のモデルよりも汎化性能が高いことが知られています。

2.  **共適応の抑制**:
    ニューロンは、他の特定のニューロンの存在を前提として学習を進める「共適応（co-adaptation）」を起こすことがあります。これは、あるニューロンのミスを他のニューロンが補うような、過度に複雑な協調関係を生み出し、過学習の原因となります。
    Dropoutは、どのニューロンがドロップされるか分からない状況を作り出すことで、各ニューロンが他の特定のニューロンに依存せず、単独でも頑健な特徴を学習するように促します。これにより、共適応が抑制され、汎化性能が向上します。

## 4. 勾配クリッピング (Gradient Clipping)

勾配クリッピングは、正則化とは少し目的が異なりますが、学習プロセスを安定させるための非常に重要な実践的テクニックです。
特にRNN（再帰型ニューラルネットワーク）の学習では、BPTTの過程で勾配が指数関数的に増加する**勾配爆発** (**Gradient Exploding**)という問題が発生しやすく、学習が発散してしまうことがあります。

勾配クリッピングは、この勾配爆発を防ぐためのシンプルな手法です。

### 4.1 勾配クリッピングのアルゴリズム

勾配クリッピングは、逆伝播によって計算された勾配の大きさが、あらかじめ定めた閾値を超えた場合に、勾配のベクトルを縮小して大きさをしきい値に抑える処理です。

**数式:**
パラメータ全体の勾配を連結したベクトルを $\boldsymbol{g}$ とします。
まず、勾配のL2ノルム（大きさ） $||\boldsymbol{g}||_2$ を計算します。
$$
||\boldsymbol{g}||_2 = \sqrt{\sum_{i} g_i^2}
$$
もし $||\boldsymbol{g}||_2$ が閾値 `max_norm` を超えていたら、勾配を以下のように更新します。
$$
\text{if } \|\boldsymbol{g}\|_2 > \text{max\_norm}:\quad \boldsymbol{g} \leftarrow \frac{\text{max\_norm}}{\|\boldsymbol{g}\|_2} \boldsymbol{g}
$$
この処理により、勾配ベクトルの**方向は変えずに、その大きさ（ノルム）だけを`max_norm`に制限**することができます。これにより、稀に発生する巨大な勾配によるパラメータの過剰な更新を防ぎ、学習プロセスを安定させることができます。

In [6]:
def gradient_clipping(grads, max_norm):
    """
    勾配クリッピングの実装
    """
    # 全ての勾配をフラットにして連結
    all_grads = np.concatenate([g.flatten() for g in grads])
    
    # L2ノルムを計算
    norm = np.linalg.norm(all_grads)
    
    # ノルムがしきい値を超えていれば、クリッピングを適用
    rate = max_norm / (norm + 1e-6) # ゼロ除算防止
    
    if rate < 1:
        print(f"Gradient norm = {norm:.2f} > max_norm = {max_norm}. Clipping applied.")
        clipped_grads = [g * rate for g in grads]
        return clipped_grads
    else:
        print(f"Gradient norm = {norm:.2f} <= max_norm = {max_norm}. No clipping.")
        return grads

# 簡単な実験
# 2つのパラメータに対する勾配を想定
grad1 = np.array([[1.0, 2.0], [3.0, 4.0]])
grad2 = np.array([[5.0, 6.0], [7.0, 8.0]])
grads = [grad1, grad2]
max_norm = 10.0

clipped_grads = gradient_clipping(grads, max_norm)

# クリッピング後のノルムを確認
clipped_all_grads = np.concatenate([g.flatten() for g in clipped_grads])
clipped_norm = np.linalg.norm(clipped_all_grads)
print(f"Norm after clipping: {clipped_norm:.2f}")

print("\n--- No clipping case ---")
max_norm_large = 20.0
_ = gradient_clipping(grads, max_norm_large)

Gradient norm = 14.28 > max_norm = 10.0. Clipping applied.
Norm after clipping: 10.00

--- No clipping case ---
Gradient norm = 14.28 <= max_norm = 20.0. No clipping.


## 5. まとめと考察

このノートブックでは、モデルの汎化性能を高めるためのDropoutと、学習を安定させるためのGradient Clippingについて学びました。

*   **Dropout**は、学習時にニューロンをランダムに非活性化することで、アンサンブル学習に似た効果を生み出し、ニューロン間の共適応を抑制します。これにより、モデルはより頑健な特徴を学習し、過学習を防ぐことができます。実装はInverted Dropoutが標準的で、推論時の追加処理が不要です。

*   **Gradient Clipping**は、勾配爆発を防ぐための安全装置です。勾配の大きさが一定のしきい値を超えた場合に、その方向を維持したまま大きさを制限します。これにより、学習中の突然の発散を防ぎ、特にRNNのような勾配が不安定になりやすいモデルの学習を安定させます。

これらのテクニックは、これまで学んできた他のコンポーネント（オプティマイザ、正規化層など）と組み合わせて使用することで、ディープニューラルネットワークの学習をより成功に導くための強力なツールとなります。

これで、ニューラルネットワークの主要な構成要素と学習テクニックを一通り学びました。これらの知識を基に、より複雑なアーキテクチャであるCNNやRNN、そしてTransformerへと学びを進めていく準備が整ったと言えるでしょう。